In [86]:
from pyomo.environ import *
import pandas as pd
import matplotlib.pyplot as plt

In [87]:
# CONTROL VARIABLES
batteryOn = False


# DATA
countries = ['DE', 'DK', 'SE']
techs = ['Wind', 'PV', 'Gas', 'Hydro', 'Battery']
efficiency = {'Wind': 1, 'PV': 1, 'Gas': 0.4, 'Hydro': 1, 'Battery': 0.9}
investment_percentages = {
    "Wind": 0.17380352644836272,
    "PV": 0.0018891687657430728,
    "Hydro": 0.44836272040302266,
    "Gas": 0.07556675062972291,
    "Battery": 0
}  # Rough approximations from ekonomifakta.se
discountrate = 0.05
hydro_max =33 * 10**6
percent_hydro_at_start = 0.5

# READ CSV
print("========================= START LOADING DATA =========================")
input_data = pd.read_csv('data/TimeSeries.csv', index_col=[0])
cap_max_data = pd.read_csv('data/capMax.csv', index_col=[0])  # MWh
cost_data = pd.read_csv('data/costs.csv', index_col=[0])
print("========================= DATA LOADED ===    ======================")

print(f"Total demand DK: {input_data['Load_DK'].sum()/1000}")
print(f"Total demand DE: {input_data['Load_DE'].sum()/1000}")
print(f"Total demand SE: {input_data['Load_SE'].sum()/1000}")


========================= START LOADING DATA =========================
========================= DATA LOADED ===    ======================
Total demand DK: 43606.417933
Total demand DE: 637617.0658
Total demand SE: 177332.34219999998


In [88]:
tot = 71.2+27.6+47.3+6.1+6.6
12/tot

0.07556675062972291

In [89]:
# UTILITY FUNCTIONS
def annualize_cost(tech):
    lifetime = cost_data.loc[tech]["lifetime"]
    return discountrate/(1-1/(1+discountrate)**lifetime)


def investment_cost(model):
    # return sum(max([model.prod[node, tech, t] for t in model.time])*cost_data.loc[tech]["investment_cost"]
    #     * annualize_cost(tech) for tech in model.gens for node in model.nodes)

    return sum(input_data["Load_" + node].sum() * investment_percentages[tech] * cost_data.loc[tech]["investment_cost"] * annualize_cost(tech) for tech in model.gens for node in model.nodes)




def capacity_max(model, n, g):
    capMax = {}
    if g in cap_max_data.columns:
        capMax[n, g] = float(cap_max_data[g].loc[cap_max_data.index == n])
        return 0.0, capMax[n, g]
    elif g == 'Battery' and not batteryOn:
        return 0.0, 0.0
    else:
        return 0.0, None


def demandData():
    demand = {}
    for n in model.nodes:
        for t in model.time:
            demand[n, t] = input_data.iloc[t][f"Load_{n}"]
    return demand

def get_load_factor(tech, time, node):
    """
    Checks performance of Wind and solar at the provided
    node and time
    """
    if tech != "Wind" and tech != "PV":
        return 1
    text = f"{tech}_{node}"
    return input_data.iloc[time][text]


In [90]:
print("========================= CREATE MODEL =========================")
model = ConcreteModel()

print("========================= CREATE SETS =========================")
# SETS
model.nodes = Set(initialize=countries, doc='countries')
model.time = Set(initialize=input_data.index, doc='hours')
model.gens = Set(initialize=techs, doc="Technologies")
# model.load_swe = Set(initialize=input_data.)


========================= CREATE MODEL =========================
========================= CREATE SETS =========================


In [91]:
print("========================= SET PARAMETERS =========================")
# PARAMETERS
model.demand = Param(model.nodes, model.time, initialize=demandData())
model.efficiency = Param(
    model.gens, initialize=efficiency, doc='Conversion efficiency')

========================= SET PARAMETERS =========================


In [92]:
print("========================= CREATE VARIABLES =========================")
# VARIABLES
model.prod = Var(model.nodes, model.gens, model.time,
                 domain=NonNegativeReals,
                 doc="Production")
model.capa = Var(model.nodes, model.gens,
                 bounds=capacity_max, doc='Generator cap')
model.water_level = Var(model.time, bounds=(0, hydro_max), doc="Water level of reservoir")

========================= CREATE VARIABLES =========================


In [93]:
print("========================= SET CONSTRAINTS =========================")
# CONSTRAINTS
# Production is limited by max capacity
def production_capacity_rule(model, nodes, gens, time):
    return model.prod[nodes, gens, time] <= model.capa[nodes, gens]

model.production_constraint = Constraint(model.nodes, model.gens,
                                         model.time, rule=production_capacity_rule)


# Total supply at each hours must meet the demand.
def demand_rule(model, nodes, gens, time):
    return sum([model.prod[nodes, tech, time] * get_load_factor(tech,time,nodes) * model.efficiency[tech] for tech in techs]) >= model.demand[nodes, time]

model.demand_constraint = Constraint(model.nodes, model.gens,
                                     model.time, rule=demand_rule)

# Producing electricity from hydro must decrease water level
def hydro_rule(model,time):
    if (time > 0):
        return model.water_level[time] == model.water_level[time - 1] - model.prod["SE", "Hydro", time] + input_data.iloc[time]["Hydro_inflow"]
    else:
        return model.water_level[time] == model.water_level[time] 

model.hydro_constraint = Constraint(model.time, rule=hydro_rule)


# Starting water level
model.water_start_constraint = Constraint(expr=model.water_level[0] == percent_hydro_at_start*hydro_max)

# Ending water_level
model.water_end_constraint = Constraint(expr=model.water_level[8759] >= model.water_level[0])






========================= SET CONSTRAINTS =========================


In [85]:
print("========================= SET OBJECTIVE FUNCTION =========================")

def objective_rule(model):
    return sum(model.prod[node, tech, t] * (cost_data.loc[tech]["variable_cost"] + cost_data.loc[tech]["fuel_cost"]) for node in model.nodes for tech in model.gens for t in model.time) + investment_cost(model)

# def objective_rule(model):
#     return sum(model.prod[node, tech, t] * (cost_data.loc[tech]["variable_cost"] + cost_data.loc[tech]["fuel_cost"] + cost_data.loc[tech]["investment_cost"] * annualize_cost(tech) ) for node in model.nodes for tech in model.gens for t in model.time) 


model.objective = Objective(
    rule=objective_rule, sense=minimize, doc='Objective function')

========================= SET OBJECTIVE FUNCTION =========================
    function to express ranged inequality expressions. (called from <ipython-
    input-80-c938da4b8571>:11)
ERROR: evaluating object as numeric value: prod[DE,Wind,0]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object prod[DE,Wind,0]
ERROR: Rule failed when generating expression for objective objective:
    ValueError: No value for uninitialized NumericValue object prod[DE,Wind,0]
ERROR: Constructing component 'objective' from data=None failed: ValueError:
    No value for uninitialized NumericValue object prod[DE,Wind,0]


ValueError: No value for uninitialized NumericValue object prod[DE,Wind,0]

In [54]:
from pyomo.opt import SolverFactory
import pyomo.environ
import pandas as pd

opt = SolverFactory("gurobi_direct")
opt.options["threads"] = 4
print("========================= SOLVING MODEL =========================")

results = opt.solve(model, tee=True)

results.write()

========================= SOLVING MODEL =========================
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown_copy
  Lower bound: 64780056648335.36
  Upper bound: 64780056648335.36
  Number of objectives: 1
  Number of constraints: 262800
  Number of variables: 140175
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 140175
  Number of nonzeros: 855640
  Sense: 1
  Number of solutions: 1
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Name: Gurobi 9.11
  Status: ok
  Wallclock time: 0.39863014221191406
  Termination condition:

In [55]:
import pprint

# Reading output - example
capTot = {}
for node in model.nodes:
    for tech in model.gens:
        for t in model.time:
            if not (node, tech) in capTot:
                capTot[node, tech] = model.prod[node, tech, t].value * get_load_factor(tech,t,node) * model.efficiency[tech] /1e3  # GW
            else:
                capTot[node, tech] += model.prod[node, tech, t].value * get_load_factor(tech,t,node) * model.efficiency[tech] /1e3

costTot = value(model.objective) / 1e6  # Million EUR
pprint.pprint(capTot)
print(f"Total cost: {costTot}")

{('DE', 'Battery'): 0.0,
 ('DE', 'Gas'): 558925.1148000011,
 ('DE', 'Hydro'): 0.0,
 ('DE', 'PV'): 69354.0432,
 ('DE', 'Wind'): 9337.907799999997,
 ('DK', 'Battery'): 0.0,
 ('DK', 'Gas'): 39149.52411499992,
 ('DK', 'Hydro'): 0.0,
 ('DK', 'PV'): 3621.781241000001,
 ('DK', 'Wind'): 835.1125769999994,
 ('SE', 'Battery'): 0.0,
 ('SE', 'Gas'): 50841.09110000008,
 ('SE', 'Hydro'): 122197.69239999996,
 ('SE', 'PV'): 1839.646400000002,
 ('SE', 'Wind'): 2453.912299999998}
Total cost: 64780056.64833544


With hydro rules
{('DE', 'Battery'): 0.0,
 ('DE', 'Gas'): 558925.1148000011,
 ('DE', 'Hydro'): 0.0,
 ('DE', 'PV'): 69354.0432,
 ('DE', 'Wind'): 9337.907799999997,
 ('DK', 'Battery'): 0.0,
 ('DK', 'Gas'): 39149.52411499992,
 ('DK', 'Hydro'): 0.0,
 ('DK', 'PV'): 3621.781241000001,
 ('DK', 'Wind'): 835.1125769999994,
 ('SE', 'Battery'): 0.0,
 ('SE', 'Gas'): 99095.99566000047,
 ('SE', 'Hydro'): 65001.188239999225,
 ('SE', 'PV'): 7019.6464000000005,
 ('SE', 'Wind'): 6215.511900000002}
Total cost: 69836710.0270978